# 라이브러리 불러오기

In [2]:
import pandas as pd 
import numpy as np

# 시각화 준비
import matplotlib
import matplotlib.pyplot as plt

# Jupyter Notebook 내부에 그래프를 출력하도록 설정
%matplotlib inline

# Seaborn은 Matplotlib을 기반으로 다양한 색상 테마와 통계용 차트 등의 기능을 추가한 시각화 패키지
import seaborn as sns

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


# 한글 깨지는거 방지하기
from matplotlib import font_manager, rc

import platform
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
matplotlib.rcParams['axes.unicode_minus'] = False   
#그래프에서 마이너스 기호가 표시되도록 하는 설정

# datetime 객체 추가
from datetime import datetime, timedelta

# train_schedule 불러와서 train data 정제 -> train

In [3]:
# pred에서 한 작업을 똑같이 진행

train = pd.read_csv("../input/AFSNT.CSV", engine = "python", encoding ='CP949')
print('Dataframe dimensions:', train.shape)
print(train.head())

train.columns = ['년','월','일', '요일', '공항', '상대공항', '항공사', '항공편', '식별번호', '출도착',
             '부정기편','계획', '실제', '지연', '지연사유', '결항', '결항사유']
recollist = ['년','월','일', '요일', '공항', '출도착', '상대공항', '항공사', '항공편', '식별번호', 
              '계획', '실제', '지연', '지연사유', '결항', '결항사유', '부정기편']
train = train[recollist]


transairport = {'ARP1':'김포', 'ARP2':'김해', 'ARP3': '제주', 'ARP4':'대구', 'ARP5': '울산',
           'ARP6':'청주', 'ARP7':'무안', 'ARP8':'광주', 'ARP9':'여수', 'ARP10':'양양',
           'ARP11':'포항', 'ARP12':'사천', 'ARP13':'군산', 'ARP14':'원주', 'ARP15':'인천'}
train['공항'] = train['공항'].apply(lambda x: transairport[x])
train['상대공항'] = train['상대공항'].apply(lambda x: transairport[x])
train["출도착"] = train["출도착"].apply(lambda x: "출발" if x=="D" else x)
train["출도착"] = train["출도착"].apply(lambda x: "도착" if x=="A" else x)
#df = df.iloc[df['항공사']!= ['C', 'D', 'E', 'G', 'K'],]
transairline = {'A':'아시아나', 'B':'에어부산', 'C': '전일본항공ANA', 'D':'에어서울', 'E': '불명',
           'F':'이스타', 'G':'일본항공', 'H':'제주항공', 'I':'진에어', 'J':'대한항공',
           'K':'타이완', 'L':'티웨이', 'M':'신규'}
train['항공사'] = train['항공사'].apply(lambda x: transairline[x])
# train.to_pickle("train.pickle")
train.head(5)

Dataframe dimensions: (987709, 17)
   SDT_YY  SDT_MM  SDT_DD SDT_DY   ARP   ODP FLO    FLT       REG AOD IRR  \
0    2017       1       1      일  ARP3  ARP6   J  J1955  SEw3NzE4   D   N   
1    2017       1       1      일  ARP3  ARP6   J  J1954  SEw3NzE4   A   N   
2    2017       1       1      일  ARP3  ARP6   J  J1956  SEw3NzE4   A   N   
3    2017       1       1      일  ARP3  ARP6   J  J1957  SEw3NzE4   D   N   
4    2017       1       1      일  ARP3  ARP6   J  J1958  SEw3NzE4   A   N   

     STT    ATT DLY  DRR CNL  CNR  
0  10:05  10:32   N  NaN   N  NaN  
1   9:30   9:31   N  NaN   N  NaN  
2  12:45  13:03   N  NaN   N  NaN  
3  13:25  14:09   Y  C02   N  NaN  
4  16:10  16:31   N  NaN   N  NaN  


,년,월,일,요일,공항,출도착,상대공항,항공사,항공편,식별번호,계획,실제,지연,지연사유,결항,결항사유,부정기편
0,2017,1,1,일,제주,출발,청주,대한항공,J1955,SEw3NzE4,10:05,10:32,N,NaN,N,NaN,N
1,2017,1,1,일,제주,도착,청주,대한항공,J1954,SEw3NzE4,9:30,9:31,N,NaN,N,NaN,N
2,2017,1,1,일,제주,도착,청주,대한항공,J1956,SEw3NzE4,12:45,13:03,N,NaN,N,NaN,N
3,2017,1,1,일,제주,출발,청주,대한항공,J1957,SEw3NzE4,13:25,14:09,Y,C02,N,NaN,N
4,2017,1,1,일,제주,도착,청주,대한항공,J1958,SEw3NzE4,16:10,16:31,N,NaN,N,NaN,N


In [124]:
train.shape

(987709, 17)

# train 결항 제거

In [125]:

# def del_cancle(row):
#     if row['결항'] == 'Y':
#         print(row)
#         del row
    
# train.apply(del_cancle, axis = 1)

train = train[train.결항 == "N"]
train.shape

(979450, 17)

In [126]:
# 계획, 실제 datetime으로 변환
# 시간값만 뽑은 칼럼 만들고 +1일 할 것 찾아서 치환
# 출발일 경우 x_차이 col에 대입
# 도착일 경우 y_차이 col에 대입
# x_차이 > 30 x_차이 < -30 인데 N라고 한것들 제외
# y_차이 > 30 y_ckdl < -30인데 N라고 한것들 제외
# x_차이 <= 30 and x_차이 >= -30 인데 Y라고 한것들 제외
# y_차이 <= 30 and y_차이 >= -30 인데 Y라고 한것들 제외

# x_차이, y_차이, time_exp, time_act 칼럼 지우기
# 계획, 실제 time만 가져와서 대입

In [127]:
train['계획'] = pd.to_datetime(train['계획'])
train['실제'] = pd.to_datetime(train['실제'])
print(train.head())

      년  월  일 요일  공항 출도착 상대공항   항공사    항공편      식별번호                  계획  \
0  2017  1  1  일  제주  출발   청주  대한항공  J1955  SEw3NzE4 2019-09-07 10:05:00   
1  2017  1  1  일  제주  도착   청주  대한항공  J1954  SEw3NzE4 2019-09-07 09:30:00   
2  2017  1  1  일  제주  도착   청주  대한항공  J1956  SEw3NzE4 2019-09-07 12:45:00   
3  2017  1  1  일  제주  출발   청주  대한항공  J1957  SEw3NzE4 2019-09-07 13:25:00   
4  2017  1  1  일  제주  도착   청주  대한항공  J1958  SEw3NzE4 2019-09-07 16:10:00   

                   실제 지연 지연사유 결항 결항사유 부정기편  
0 2019-09-07 10:32:00  N  NaN  N  NaN    N  
1 2019-09-07 09:31:00  N  NaN  N  NaN    N  
2 2019-09-07 13:03:00  N  NaN  N  NaN    N  
3 2019-09-07 14:09:00  Y  C02  N  NaN    N  
4 2019-09-07 16:31:00  N  NaN  N  NaN    N  


In [128]:
train['time_exp'] = train['계획'].apply(lambda x : x.time().hour)
train['time_act'] = train['실제'].apply(lambda x : x.time().hour)

def nextday_time_mapping(row):
  if (row['time_exp'] >= 20) and (row['time_act'] >=0) and (row['time_act'] <= 5):
    print(row)
    return row['실제'] + timedelta(days = +1)
  else:
    return row['실제']

train.head()


,년,월,일,요일,공항,출도착,상대공항,항공사,항공편,식별번호,계획,실제,지연,지연사유,결항,결항사유,부정기편,time_exp,time_act
0,2017,1,1,일,제주,출발,청주,대한항공,J1955,SEw3NzE4,2019-09-07 10:05:00,2019-09-07 10:32:00,N,NaN,N,NaN,N,10,10
1,2017,1,1,일,제주,도착,청주,대한항공,J1954,SEw3NzE4,2019-09-07 09:30:00,2019-09-07 09:31:00,N,NaN,N,NaN,N,9,9
2,2017,1,1,일,제주,도착,청주,대한항공,J1956,SEw3NzE4,2019-09-07 12:45:00,2019-09-07 13:03:00,N,NaN,N,NaN,N,12,13
3,2017,1,1,일,제주,출발,청주,대한항공,J1957,SEw3NzE4,2019-09-07 13:25:00,2019-09-07 14:09:00,Y,C02,N,NaN,N,13,14
4,2017,1,1,일,제주,도착,청주,대한항공,J1958,SEw3NzE4,2019-09-07 16:10:00,2019-09-07 16:31:00,N,NaN,N,NaN,N,16,16


In [129]:
today_day = pd.to_datetime('today').day
train['실제'] = train.apply(nextday_time_mapping, axis = 1)


년                          2017
월                            10
일                             2
요일                            월
공항                           제주
출도착                          도착
상대공항                         청주
항공사                         진에어
항공편                       I1559
식별번호                   SEw3NTYz
계획          2019-09-07 20:25:00
실제          2019-09-07 00:05:00
지연                            N
지연사유                        NaN
결항                            N
결항사유                        NaN
부정기편                          N
time_exp                     20
time_act                      0
Name: 354335, dtype: object
년                          2018
월                             2
일                             5
요일                            월
공항                           제주
출도착                          출발
상대공항                         인천
항공사                        대한항공
항공편                       J1707
식별번호                   SEw3NzY0
계획          2019-09-07 23:05:00
실제          

년                          2018
월                             5
일                            18
요일                            금
공항                           제주
출도착                          도착
상대공항                         김포
항공사                         티웨이
항공편                       L1735
식별번호                   SEw4MjM1
계획          2019-09-07 23:26:00
실제          2019-09-07 00:28:00
지연                            Y
지연사유                        C02
결항                            N
결항사유                        NaN
부정기편                          N
time_exp                     23
time_act                      0
Name: 772609, dtype: object
년                          2019
월                             2
일                            21
요일                            목
공항                           무안
출도착                          도착
상대공항                         제주
항공사                        아시아나
항공편                       A1168
식별번호                   SEw3NzM3
계획          2019-09-07 20:50:00
실제          

In [130]:
def check(row):
    if row['실제'].day == today_day+1:
        print(row)
        
train.apply(check, axis = 1)

년                          2017
월                            10
일                             2
요일                            월
공항                           제주
출도착                          도착
상대공항                         청주
항공사                         진에어
항공편                       I1559
식별번호                   SEw3NTYz
계획          2019-09-07 20:25:00
실제          2019-09-08 00:05:00
지연                            N
지연사유                        NaN
결항                            N
결항사유                        NaN
부정기편                          N
time_exp                     20
time_act                      0
Name: 354335, dtype: object
년                          2018
월                             2
일                             5
요일                            월
공항                           제주
출도착                          출발
상대공항                         인천
항공사                        대한항공
항공편                       J1707
식별번호                   SEw3NzY0
계획          2019-09-07 23:05:00
실제          

년                          2019
월                             2
일                            21
요일                            목
공항                           무안
출도착                          도착
상대공항                         제주
항공사                        아시아나
항공편                       A1168
식별번호                   SEw3NzM3
계획          2019-09-07 20:50:00
실제          2019-09-08 05:45:00
지연                            N
지연사유                        NaN
결항                            N
결항사유                        NaN
부정기편                          N
time_exp                     20
time_act                      5
Name: 901736, dtype: object
년                          2019
월                             5
일                            18
요일                            토
공항                           대구
출도착                          도착
상대공항                         제주
항공사                         티웨이
항공편                       L1810
식별번호                   SEw4MjY4
계획          2019-09-07 22:10:00
실제          

0         None
1         None
2         None
3         None
4         None
5         None
6         None
7         None
8         None
9         None
10        None
11        None
12        None
13        None
14        None
15        None
16        None
17        None
18        None
19        None
20        None
21        None
22        None
23        None
24        None
25        None
26        None
27        None
28        None
29        None
          ... 
987679    None
987680    None
987681    None
987682    None
987683    None
987684    None
987685    None
987686    None
987687    None
987688    None
987689    None
987690    None
987691    None
987692    None
987693    None
987694    None
987695    None
987696    None
987697    None
987698    None
987699    None
987700    None
987701    None
987702    None
987703    None
987704    None
987705    None
987706    None
987707    None
987708    None
Length: 979450, dtype: object

In [131]:
train["차이"] = train["실제"] - train["계획"]
train["차이_minus"] = train["계획"] - train["실제"]


train['차이'][train['차이'].astype('timedelta64[m]') > 0 ] =   train['차이'].astype('timedelta64[m]') 
train['차이'][train['차이_minus'].astype('timedelta64[m]') > 0 ] = 0 - train['차이_minus'].astype('timedelta64[m]') 

train.head(100)


/Users/Choiwansik/anaconda3/envs/kagglecom/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/Choiwansik/anaconda3/envs/kagglecom/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,년,월,일,요일,공항,출도착,상대공항,항공사,항공편,식별번호,...,실제,지연,지연사유,결항,결항사유,부정기편,time_exp,time_act,차이,차이_minus
0,2017,1,1,일,제주,출발,청주,대한항공,J1955,SEw3NzE4,...,2019-09-07 10:32:00,N,NaN,N,NaN,N,10,10,27.0,-1 days +23:33:00
1,2017,1,1,일,제주,도착,청주,대한항공,J1954,SEw3NzE4,...,2019-09-07 09:31:00,N,NaN,N,NaN,N,9,9,1.0,-1 days +23:59:00
2,2017,1,1,일,제주,도착,청주,대한항공,J1956,SEw3NzE4,...,2019-09-07 13:03:00,N,NaN,N,NaN,N,12,13,18.0,-1 days +23:42:00
3,2017,1,1,일,제주,출발,청주,대한항공,J1957,SEw3NzE4,...,2019-09-07 14:09:00,Y,C02,N,NaN,N,13,14,44.0,-1 days +23:16:00
4,2017,1,1,일,제주,도착,청주,대한항공,J1958,SEw3NzE4,...,2019-09-07 16:31:00,N,NaN,N,NaN,N,16,16,21.0,-1 days +23:39:00
5,2017,1,1,일,제주,출발,청주,대한항공,J1959,SEw3NTk5,...,2019-09-07 17:21:00,Y,C02,N,NaN,N,16,17,36.0,-1 days +23:24:00
6,2017,1,1,일,제주,도착,청주,대한항공,J1960,SEw3NTk5,...,2019-09-07 19:43:00,N,NaN,N,NaN,N,19,19,13.0,-1 days +23:47:00
7,2017,1,1,일,제주,출발,청주,대한항공,J1961,SEw3NTk5,...,2019-09-07 20:52:00,N,NaN,N,NaN,N,20,20,17.0,-1 days +23:43:00
8,2017,1,1,일,김해,도착,제주,대한항공,J1015,SEw3NzA2,...,2019-09-07 17:03:00,N,NaN,N,NaN,N,17,17,-2.0,00:02:00
9,2017,1,1,일,김포,출발,제주,대한항공,J1242,SEw3NzA2,...,2019-09-07 20:36:00,N,NaN,N,NaN,N,20,20,11.0,-1 days +23:49:00


In [132]:
train.columns

Index(['년', '월', '일', '요일', '공항', '출도착', '상대공항', '항공사', '항공편', '식별번호', '계획',
       '실제', '지연', '지연사유', '결항', '결항사유', '부정기편', 'time_exp', 'time_act', '차이',
       '차이_minus'],
      dtype='object')

In [133]:
idx_1 = train[((train['차이'] > 30) | (train['차이'] < -30.0)) & (train['지연'] == 'N')].index
train = train.drop(idx_1,axis = 0)
idx_2 = train[((train['차이'] <= 30) & (train['차이'] >= -30.0)) & (train['지연'] == 'Y')].index
train = train.drop(idx_2,axis = 0)
train.shape

(978734, 21)

In [134]:
remove_variable = ['time_exp','time_act','차이','차이_minus']
train.drop(remove_variable, axis = 1, inplace = True)
train.head()

,년,월,일,요일,공항,출도착,상대공항,항공사,항공편,식별번호,계획,실제,지연,지연사유,결항,결항사유,부정기편
0,2017,1,1,일,제주,출발,청주,대한항공,J1955,SEw3NzE4,2019-09-07 10:05:00,2019-09-07 10:32:00,N,NaN,N,NaN,N
1,2017,1,1,일,제주,도착,청주,대한항공,J1954,SEw3NzE4,2019-09-07 09:30:00,2019-09-07 09:31:00,N,NaN,N,NaN,N
2,2017,1,1,일,제주,도착,청주,대한항공,J1956,SEw3NzE4,2019-09-07 12:45:00,2019-09-07 13:03:00,N,NaN,N,NaN,N
3,2017,1,1,일,제주,출발,청주,대한항공,J1957,SEw3NzE4,2019-09-07 13:25:00,2019-09-07 14:09:00,Y,C02,N,NaN,N
4,2017,1,1,일,제주,도착,청주,대한항공,J1958,SEw3NzE4,2019-09-07 16:10:00,2019-09-07 16:31:00,N,NaN,N,NaN,N


In [135]:
train['계획'] = train['계획'].apply(lambda x : str(x)[11:16])
train['실제'] = train['실제'].apply(lambda x : str(x)[11:16])
train.head()

,년,월,일,요일,공항,출도착,상대공항,항공사,항공편,식별번호,계획,실제,지연,지연사유,결항,결항사유,부정기편
0,2017,1,1,일,제주,출발,청주,대한항공,J1955,SEw3NzE4,10:05,10:32,N,NaN,N,NaN,N
1,2017,1,1,일,제주,도착,청주,대한항공,J1954,SEw3NzE4,09:30,09:31,N,NaN,N,NaN,N
2,2017,1,1,일,제주,도착,청주,대한항공,J1956,SEw3NzE4,12:45,13:03,N,NaN,N,NaN,N
3,2017,1,1,일,제주,출발,청주,대한항공,J1957,SEw3NzE4,13:25,14:09,Y,C02,N,NaN,N
4,2017,1,1,일,제주,도착,청주,대한항공,J1958,SEw3NzE4,16:10,16:31,N,NaN,N,NaN,N


In [136]:
train.shape
train.head()

,년,월,일,요일,공항,출도착,상대공항,항공사,항공편,식별번호,계획,실제,지연,지연사유,결항,결항사유,부정기편
0,2017,1,1,일,제주,출발,청주,대한항공,J1955,SEw3NzE4,10:05,10:32,N,NaN,N,NaN,N
1,2017,1,1,일,제주,도착,청주,대한항공,J1954,SEw3NzE4,09:30,09:31,N,NaN,N,NaN,N
2,2017,1,1,일,제주,도착,청주,대한항공,J1956,SEw3NzE4,12:45,13:03,N,NaN,N,NaN,N
3,2017,1,1,일,제주,출발,청주,대한항공,J1957,SEw3NzE4,13:25,14:09,Y,C02,N,NaN,N
4,2017,1,1,일,제주,도착,청주,대한항공,J1958,SEw3NzE4,16:10,16:31,N,NaN,N,NaN,N


In [137]:
train.to_csv('train_preprocessed.csv', encoding = "CP949")